## Running Spike interface on data preprocessed with CatGT

The processed data will be saved to: ``<destination parent>/<run_name>_g<gate>``. Open it up in the SpikeGLX viewer and look for any problems. Is the background subtraction sufficient? Are any major artifacts removed?

After checking that the preprocessed data looks OK, move to sorting a single shank, using spike interface


Check the instructions [**here**](https://spikeinterface.readthedocs.io/en/0.93.0/overview.html).

__spikeinterface__ is installed in the npix-analysis environment on the _course computers_; the installation instructions are [**here**](https://spikeinterface.readthedocs.io/en/0.93.0/installation.html).

Be sure to checkout the other [tutorials](https://spikeinterface.readthedocs.io/en/0.93.0/getting_started/plot_getting_started.html#).

---

We will build and run a pipeline on this notebook, it is based on the [how to guide](https://spikeinterface.readthedocs.io/en/latest/how_to/analyze_neuropixels.html). 


In [ ]:
import os
import subprocess

os_str='Windows'

# set the parent path for the software on your computer
software_download_path = r'C:\Users\labadmin\Desktop\software_downloads'

# parent folder of the data
data_parent = r'C:\Users\labadmin\Desktop\test_data'

# run name for the acquired data (don't include any gate info)
run_name = 'SC035_010720_ex'

# gate index (for myrun_g0_t0.imec0.ap.bin, the gate = 0)
gate = 0

# destination parent
dest_parent = r'C:\Users\labadmin\Desktop\test_data\output'

# time to analyze and sort 
maxsecs = 600

# extraction parameters for stimulus times. parameters: stream type(js), stream index(ip), channel in file, threshold1, threshold2, duration
obx_ex_str = '-xa=1,0,1,1,3,50'

dest_dir = os.path.join(dest_parent,f'{run_name}_g{gate}')
if not os.path.isdir(dest_dir):
    os.mkdir(dest_dir)

cmd_parts = list()
cmd_parts.append(os.path.join(software_download_path,'ecephys_course/CatGT-win/CatGT.exe'))
cmd_parts.append(f'-dir={data_parent} -run={run_name} -g={gate} -t=0,0 -prb_fld')
cmd_parts.append(f'-prb=0 -ap')
cmd_parts.append(f'-apfilter=butter,12,300,10000 -gblcar -gfix=0.40,0.10,0.02')
#cmd_parts.append(f'-ob {obx_ex_str}')
cmd_parts.append(f'-maxsecs={maxsecs}')
cmd_parts.append(f'-out_prb_fld -no_catgt_fld -dest={dest_dir}')
                        

catGT_cmd = ' '        # use space as the separator for the command parts
catGT_cmd = catGT_cmd.join(cmd_parts[1:len(cmd_parts)]) # these are the parameters
if os_str=='linux':
    # enclose the params in single quotes, so curly braces will not be interpreted by Linux
    catGT_cmd = f"{cmd_parts[0]} '{catGT_cmd}'"
else:
    catGT_cmd = f"{cmd_parts[0]} {catGT_cmd}"

print(catGT_cmd)
subprocess.Popen(catGT_cmd,shell='False').wait()
print('CatGT complete, check CatGT.log in notebook folder for errors.')


In [ ]:
import spikeinterface.full as si
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

# parameters -- these only pertain to the sorter, because the data is already preprocessed.
# in this example, we keep the filtering and whitening in Kilosort 4. This is to keep the native Ks4 whitening (to avoid scaling problems)

ks4_params = si.get_default_sorter_params('kilosort4')
ks4_params['do_CAR'] = False # skip CAR in kilosort
job_kwargs = dict(n_jobs=-1, chunk_duration='1s', progress_bar=True) # how to chunk and process data

sort_folder = os.path.join(dest_dir,f'ks4_out')  # this folder is created by KS4; existing folder cannot be overwritten

# load spikeglx data, pointing to the destination folder created above
stream_names, stream_ids = si.get_neo_streams('spikeglx', dest_dir)

print(f'There are streams {stream_names} in folder {dest_dir}')


In [ ]:
# SET the stream parameter for the file you want to sort here

selected_stream = 'imec0.ap'

print(f'All Kilosort4 parameters are: ')
for k in ks4_params.keys():
    print(f'   - {k}: {ks4_params[k]}')

In [ ]:
# sort the stream for the selected shank

rec = si.read_spikeglx(dest_dir, stream_name=selected_stream, load_sync_channel=False)

# run ks4
sorting = si.run_sorter('kilosort4', rec, folder=sort_folder,
                        docker_image=False, verbose=True, **ks4_params)
sorting = si.read_sorter_folder(sort_folder)

# phy output from ks4 is saved to sort_folder/sorter_output

In [ ]:
# calculate average waveforms and metrics.

sorting = si.read_sorter_folder(sort_folder)

analyzer = si.create_sorting_analyzer(sorting, rec, sparse=True, format="memory")
# compute waveforms 
analyzer.compute("random_spikes", method="uniform", max_spikes_per_unit=500)
analyzer.compute("waveforms",  ms_before=1.5, ms_after=1.5, **job_kwargs)
analyzer.compute("templates", operators=["average", "median", "std"])
analyzer.compute("noise_levels")
analyzer.compute("correlograms")
analyzer.compute("unit_locations")
analyzer.compute("spike_amplitudes", **job_kwargs)

# quality metrics
metric_names=['firing_rate', 'presence_ratio', 'snr', 'isi_violation', 'amplitude_cutoff']
metrics = si.compute_quality_metrics(analyzer, metric_names=metric_names)

# all of the analyzer computed quantitites are saved in sort_folder/analyzer/extensions
analyzer_saved = analyzer.save_as(folder=os.path.join(sort_folder, "analyzer"), format="binary_folder")
